In [1]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

main_directory = 'C:/New folder/animals'

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255
)

target_size = (224, 224)

batch_size = 32

train_generator = datagen.flow_from_directory(
    main_directory,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)

val_generator = datagen.flow_from_directory(
    main_directory,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    subset='validation'
)


Found 5400 images belonging to 90 classes.
Found 0 images belonging to 90 classes.


In [9]:
import os
import shutil
from sklearn.model_selection import train_test_split

main_directory = 'C:/New folder/animals'

train_directory = 'C:/New folder/trainData'

val_directory = 'C:/New folder/validationData'

test_size = 0.2

os.makedirs(train_directory, exist_ok=True)
os.makedirs(val_directory, exist_ok=True)

subdirectories = [f.name for f in os.scandir(main_directory) if f.is_dir()]

for subdirectory in subdirectories:
    subdirectory_path = os.path.join(main_directory, subdirectory)
    train_subdirectory = os.path.join(train_directory, subdirectory)
    val_subdirectory = os.path.join(val_directory, subdirectory)
    
    os.makedirs(train_subdirectory, exist_ok=True)
    os.makedirs(val_subdirectory, exist_ok=True)
    
    image_files = [f.name for f in os.scandir(subdirectory_path) if f.is_file()]
    
    train_files, val_files = train_test_split(image_files, test_size=test_size, random_state=42)
    
    for train_file in train_files:
        src = os.path.join(subdirectory_path, train_file)
        dst = os.path.join(train_subdirectory, train_file)
        shutil.copy(src, dst)

    for val_file in val_files:
        src = os.path.join(subdirectory_path, val_file)
        dst = os.path.join(val_subdirectory, val_file)
        shutil.copy(src, dst)


In [10]:
train_generator = datagen.flow_from_directory(
    'C:/New folder/trainData',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    'C:/New folder/validationData',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 4320 images belonging to 90 classes.
Found 1080 images belonging to 90 classes.


In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


In [12]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

model.add(Dense(90, activation='softmax'))


In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples//validation_generator.batch_size,
    epochs=10
)

test_loss, test_acc = model.evaluate(test_images, test_labels)


Epoch 1/10
135/135 [==============================] - 314s 2s/step - loss: 4.9850 - accuracy: 0.0100 - val_loss: 4.4915 - val_accuracy: 0.0085
Epoch 2/10
135/135 [==============================] - 224s 2s/step - loss: 4.4906 - accuracy: 0.0120 - val_loss: 4.4774 - val_accuracy: 0.0142
Epoch 3/10
135/135 [==============================] - 185s 1s/step - loss: 4.4189 - accuracy: 0.0167 - val_loss: 4.4202 - val_accuracy: 0.0256
Epoch 4/10
135/135 [==============================] - 186s 1s/step - loss: 4.3565 - accuracy: 0.0299 - val_loss: 4.3735 - val_accuracy: 0.0360
Epoch 5/10
135/135 [==============================] - 197s 1s/step - loss: 4.3039 - accuracy: 0.0329 - val_loss: 4.3753 - val_accuracy: 0.0256
Epoch 6/10
135/135 [==============================] - 185s 1s/step - loss: 4.2516 - accuracy: 0.0331 - val_loss: 4.3026 - val_accuracy: 0.0436
Epoch 7/10
135/135 [==============================] - 350s 3s/step - loss: 4.2239 - accuracy: 0.0382 - val_loss: 4.2658 - val_accuracy: 0.0360

NameError: name 'test_images' is not defined

In [17]:
test_data_dir = 'C:/New folder/testData'
test_generator = datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)
test_loss, test_acc = model.evaluate(test_generator)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)
 

Found 810 images belonging to 90 classes.
26/26 [==============================] - 20s 801ms/step - loss: 4.1043 - accuracy: 0.0519
Test Loss: 4.1043171882629395
Test Accuracy: 0.051851850003004074
